# Library

In [164]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    GPT2Config,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    TFGPT2Model,
    Trainer,
    TrainingArguments,
)

# GPT2

reference: [GPT2](https://wikidocs.net/184363) <br>
paper: [GPT2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) 

## Generation

In [10]:
# 'skt/kogpt2-base-v2' 모델을 불러와서 모델 객체를 생성합니다.
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# 'skt/kogpt2-base-v2'에 맞는 토크나이저를 불러와서 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

In [48]:
# 입력 문장을 토크나이저를 사용하여 인코딩합니다.
# 'return_tensors' 매개변수를 'pt'로 설정하여 PyTorch 텐서 형식으로 반환합니다.
encoded = tokenizer('파이썬을 잘 학습하기 위해서는', return_tensors='pt')

# 인코딩된 텐서를 출력합니다.
encoded

{'input_ids': tensor([[27752,  7918,  8137,  9443, 12384,  9154, 11357]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [49]:
# 모델을 사용하여 텍스트를 생성합니다.
output = model.generate(
    # 인코딩된 입력 ID를 모델에 전달합니다.
    encoded.get('input_ids'),
    # 생성할 최대 길이를 설정합니다.
    max_length=256,
    # 반복되는 단어를 억제하기 위한 패널티 값을 설정합니다.
    repetition_penalty=2.0,
    # 캐시 사용을 설정하여 이전의 결과를 활용합니다.
    use_cache=True,
)

# 생성된 텍스트를 디코딩하여 사람이 읽을 수 있는 형식으로 변환합니다.
print(tokenizer.decode(output[0]))

파이썬을 잘 학습하기 위해서는 먼저 자신의 생각을 논리적으로 표현하는 능력을 길러야 한다.
논리적인 사고는 논리적 사고를 통해 문제를 해결하는 능력이다.
이러한 능력은 문제 해결에 필요한 핵심 역량인 것이다.
따라서 논술은 단순히 암기하는 것이 아니라 다양한 경험을 바탕으로 한 종합적인 사고와 함께 해야 하는 중요한 과정이다.
논술에서 가장 중요하게 다루는 것은 바로 ‘논리적’이다.
즉, 논술에서는 주어진 문제에 대한 정확한 이해와 더불어 문제의 핵심을 파악하는 데 초점을 맞추어야 한다는 말이다.
또한, 이러한 과정을 거쳐야만 비로소 자신이 생각하는 바를 정확하게 이해할 수 있다.
그렇다면 어떻게 하면 효과적인 대안이 될까?
우선 어떤 방법으로 해결할 것인가?
먼저 제시된 문제와 관련된 구체적인 사례를 중심으로 살펴보자.
첫째, 제시문을 읽고 그 내용을 요약해 보자.
둘째로, 글의 전체적인 흐름을 파악하자.
셋째로 글을 읽는 사람의 입장에서 생각해보자.</d> 지난달 30일 오후 서울 강남구 삼성동 코엑스 컨벤션홀의 대형 스크린에는 '2018 평창 동계올림픽' 홍보 포스터가 내걸렸다.
평창동계 올림픽은 오는 2022년 2월 9일부터 3월 8일까지 열흘간 열린다.
홍보 포스터를 본 누리꾼들은 "아직까지도 많은 분들이 관심을 갖고 계신 것 같다"며 "이번에도 좋은 결과가 있을 것으로 기대된다"고 입을 모았다.
이어 그는 이번 주말께 공식 홈페이


In [86]:
# 초기 입력 문장
inputs = '파이썬을 잘 학습하기 위해서는'

# 입력 문장을 토크나이저를 사용하여 인코딩합니다.
encoded = tokenizer(inputs, return_tensors='pt')

# 입력 문장의 길이가 64 이하인 동안 반복합니다.
while len(encoded.get('input_ids')[0]) < 64:
    # 모델을 사용하여 다음 토큰에 대한 출력을 생성합니다.
    output = model(**encoded)
    
    # 마지막 위치의 로짓에서 상위 k개의 인덱스를 추출합니다.
    indices = torch.topk(output.logits[0, -1], k=4).indices
    
    # 랜덤하게 선택된 인덱스를 기반으로 다음 토큰을 결정합니다.
    top1 = np.random.choice(indices)
    
    # 선택된 토큰의 ID를 디코딩하여 문자로 변환합니다.
    next_token = tokenizer.convert_ids_to_tokens([top1])[0].replace('▁', ' ')
    
    # 새로운 토큰을 기존 입력에 추가합니다.
    inputs = ''.join([inputs, next_token])
    
    # 업데이트된 입력을 다시 인코딩합니다.
    encoded = tokenizer(inputs, return_tensors='pt')

In [88]:
print(inputs)

파이썬을 잘 학습하기 위해서는 ‘학습법’이 중요하다.
이 책은 독해력과 문제 해결력, 문제 해결력, 사고력과 창의력을 키우는 학습법을 알려준다.
독서는 자기 주도학습이 아니라 학습자의 자기 주도적인 학습을 돕는 학습법이다.
이 책들은 독서를 하면서 자신의 생각을 정리하고 스스로 문제를 해결해 나가는 과정을 담았다는 점이 눈길을


## Fine-tuning

### Chatbot

In [94]:
dataset = load_dataset(
    'bitext/Bitext-customer-support-llm-chatbot-training-dataset'
)

In [124]:
# Hugging Face의 Transformers 라이브러리에서 OpenAI의 GPT-2 토크나이저를 불러옵니다.
tokenizer = AutoTokenizer.from_pretrained(
    'openai-community/gpt2',
)

# 시작 토큰(BOS, Beginning of Sequence)으로 사용할 토큰을 설정합니다.
tokenizer.bos_token='</s>'

# 종료 토큰(EOS, End of Sequence)으로 사용할 토큰을 설정합니다.
tokenizer.eos_token='</s>'

# 패딩 토큰(PAD, Padding)으로 사용할 토큰을 설정합니다.
tokenizer.pad_token='<pad>'


In [126]:
# 전체 데이터셋에서 훈련 데이터의 일부를 선택합니다.
# 여기서는 첫 2000개의 샘플을 선택하여 훈련 데이터셋을 만듭니다.
train_dataset = dataset['train'].select(range(2000))

In [127]:
def preprocessing(row):
    sentence = ''.join([
        '<usr> ',
        row['instruction'],
        ' <sys> ',
        row['response'],
    ])
    sentence = ''.join(['</s>', sentence, '</s>'])
    
    tokenized = tokenizer(
        sentence,
        padding='max_length',
        max_length=256,
        return_tensors='pt',
        truncation=True,
    )

    return tokenized

In [76]:
## list로 데이터가 들어가서 동작 안 됨
train_dataset = train_dataset.map(preprocessing, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


TypeError: sequence item 1: expected str instance, list found

In [128]:
# 훈련 데이터셋의 각 샘플에 대해 전처리 함수를 적용하여 토크나이즈된 데이터셋을 생성합니다.
train_dataset = [
    preprocessing(train_dataset[i])  # 각 샘플에 대해 preprocessing 함수를 호출합니다.
    for i in range(len(train_dataset))  # 데이터셋의 모든 인덱스에 대해 반복합니다.
]

In [129]:
class ChatbotDataset(Dataset):
    def __init__(self, data):
        # 데이터셋 초기화: 입력 데이터를 저장합니다.
        self.data = data
    
    def __len__(self):
        # 데이터셋의 길이(샘플 수)를 반환합니다.
        return len(self.data)
    
    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 샘플을 가져옵니다.
        temp = self.data[idx]
        
        # 결과 딕셔너리를 생성합니다.
        result = {
            'input_ids': temp.get('input_ids')[0],        # 입력 ID
            'attention_mask': temp.get('attention_mask')[0],  # 주의 마스크
            'labels': temp.get('input_ids')[0]             # 학습에 사용할 라벨 (입력 ID와 동일)
        }

        return result  # 결과 딕셔너리를 반환합니다.

In [130]:
# 전처리된 훈련 데이터셋을 ChatbotDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = ChatbotDataset(train_dataset)

In [ ]:
# 모델 학습을 위한 설정을 정의합니다.
training_args = TrainingArguments(
    output_dir='./GPT2/chatbot/results/',  # 결과를 저장할 디렉토리
    eval_strategy='epoch',                   # 평가 전략: 매 에포크마다 평가
    learning_rate=2e-5,                     # 학습률
    warmup_steps=50,                        # 워밍업 단계 수
    per_device_train_batch_size=16,        # 훈련 시 배치 크기
    per_device_eval_batch_size=16,         # 평가 시 배치 크기
    num_train_epochs=1,                     # 총 훈련 에포크 수
    weight_decay=0.01,                      # 가중치 감쇠
    logging_dir='./GPT2/chatbot/logs',     # 로그를 저장할 디렉토리
)

# 미리 학습된 GPT-2 모델을 불러옵니다.
model = GPT2LMHeadModel.from_pretrained('openai-community/gpt2')

# Trainer 객체를 생성하여 모델 훈련을 준비합니다.
trainer = Trainer(
    model=model,               # 훈련할 모델
    args=training_args,       # 훈련 설정
    train_dataset=train_dataset,  # 훈련 데이터셋
    # eval_dataset=valid_dataset,   # 검증 데이터셋 (주석 처리됨)
)

# 모델 훈련을 시작합니다.
trainer.train()

# 훈련이 완료된 모델을 지정한 디렉토리에 저장합니다.
model.save_pretrained('./GPT2/chatbot')

#### 한국어 챗봇

In [135]:
data = pd.read_csv('./data/ChatBotData.csv')

In [146]:
# 데이터프레임의 각 행에 대해 Q(질문)와 A(답변)를 결합하여 새로운 열 'joined_data'를 생성합니다.
data['joined_data'] = data.apply(
    lambda x: ''.join(['</s><usr>', x['Q'], ' <sys>', x['A'], '</s>']),  # 각 질문과 답변을 포맷합니다.
    axis=1  # 행 단위로 적용합니다.
)

In [149]:
# Hugging Face의 Transformers 라이브러리에서 SKT의 KoGPT-2 토크나이저를 불러옵니다.
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# 종료 토큰(EOS, End of Sequence)으로 사용할 토큰을 설정합니다.
tokenizer.eos_token = '</s>'

# 시작 토큰(BOS, Beginning of Sequence)으로 사용할 토큰을 설정합니다.
tokenizer.bos_token = '</s>'

# 패딩 토큰(PAD, Padding)으로 사용할 토큰을 설정합니다.
tokenizer.pad_token = '<pad>'

In [154]:
# 'joined_data' 열의 각 문장을 토크나이즈하여 'tokenized_data' 열에 저장합니다.
data['tokenized_data'] = data.joined_data.apply(
    lambda x: tokenizer(
        x,                          # 토크나이즈할 입력 문장
        padding='max_length',      # 최대 길이까지 패딩합니다.
        max_length=256,            # 최대 길이를 256으로 설정합니다.
        truncation=True,           # 길이를 초과할 경우 잘라냅니다.
        return_tensors='pt',      # PyTorch 텐서 형식으로 반환합니다.
    )
)

In [162]:
from torch.utils.data import Dataset

class KorChatbotDataset(Dataset):
    def __init__(self, data):
        # 데이터셋 초기화: 입력 데이터를 저장합니다.
        self.data = data

    def __len__(self):
        # 데이터셋의 길이(샘플 수)를 반환합니다.
        return len(self.data)
    
    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 토크나이즈된 데이터를 가져옵니다.
        temp = self.data.iloc[idx].tokenized_data
        
        # 결과 딕셔너리를 생성합니다.
        result = {
            'input_ids': temp.get('input_ids')[0],        # 입력 ID
            'attention_mask': temp.get('attention_mask')[0],  # 주의 마스크
            'labels': temp.get('input_ids')[0],             # 학습에 사용할 라벨 (입력 ID와 동일)
        }

        return result  # 결과 딕셔너리를 반환합니다.

In [165]:
# 전체 데이터를 70%의 훈련 데이터와 30%의 검증 데이터로 분할합니다.
train, valid = train_test_split(data, test_size=0.3, random_state=0)

# 훈련 데이터와 검증 데이터를 각각 KorChatbotDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = KorChatbotDataset(train)
valid_dataset = KorChatbotDataset(valid)

In [ ]:
# 모델 학습을 위한 설정을 정의합니다.
training_args = TrainingArguments(
    output_dir='./GPT2/kor_chatbot/results/',  # 결과를 저장할 디렉토리
    eval_strategy='epoch',                       # 평가 전략: 매 에포크마다 평가
    learning_rate=2e-5,                         # 학습률
    warmup_steps=50,                            # 워밍업 단계 수
    per_device_train_batch_size=32,            # 훈련 시 배치 크기
    per_device_eval_batch_size=32,             # 평가 시 배치 크기
    num_train_epochs=1,                         # 총 훈련 에포크 수
    weight_decay=0.01,                          # 가중치 감쇠
    logging_dir='./GPT2/kor_chatbot/logs',     # 로그를 저장할 디렉토리
)

# 미리 학습된 KoGPT-2 모델을 불러옵니다.
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# Trainer 객체를 생성하여 모델 훈련을 준비합니다.
trainer = Trainer(
    model=model,                # 훈련할 모델
    args=training_args,        # 훈련 설정
    train_dataset=train_dataset,  # 훈련 데이터셋
    eval_dataset=valid_dataset,   # 검증 데이터셋
)

# 모델 훈련을 시작합니다.
trainer.train()

# 훈련이 완료된 모델을 지정한 디렉토리에 저장합니다.
model.save_pretrained('./GPT2/kor_chatbot')

In [172]:
# 미리 학습된 KoGPT-2 모델을 지정한 경로에서 불러옵니다.
model = GPT2LMHeadModel.from_pretrained('./model/kor_chatbot')

def generate_chat(user_input: str, topk: int = 5, max_length: int = 128):
    # 사용자 입력을 포맷하여 모델에 전달할 입력 문자열을 생성합니다.
    inputs = f'</s><usr>{user_input} <sys>'
    
    # 입력 문자열을 토크나이즈하여 텐서 형식으로 변환합니다.
    encoded = tokenizer(inputs, return_tensors='pt')

    # 최대 길이에 도달할 때까지 반복하여 다음 단어를 생성합니다.
    while len(encoded.get('input_ids')[0]) <= max_length:
        # 모델을 사용하여 다음 토큰에 대한 출력을 생성합니다.
        output = model(**encoded)
        
        # 마지막 위치의 로짓에서 상위 k개의 인덱스를 추출합니다.
        indices = torch.topk(output.logits[0, -1], k=topk).indices
        
        # 랜덤하게 선택된 인덱스를 기반으로 다음 토큰을 결정합니다.
        top1 = np.random.choice(indices)
        
        # 선택된 토큰의 ID를 디코딩하여 문자로 변환합니다.
        next_token = tokenizer.convert_ids_to_tokens([top1])[0].replace('▁', ' ')
        
        # 새로운 토큰을 기존 입력에 추가합니다.
        inputs = ''.join([inputs, next_token])
        
        # 업데이트된 입력을 다시 인코딩합니다.
        encoded = tokenizer(inputs, return_tensors='pt')
    
    return inputs  # 최종 생성된 문자열을 반환합니다.

  0%|          | 0/125 [1:26:17<?, ?it/s]


In [180]:
# 사용자 입력 '힘들어서 결혼할까봐'를 기반으로 챗봇의 응답을 생성합니다.
# 최대 길이는 32로 설정합니다.
result = generate_chat('힘들어서 결혼할까봐', max_length=32)
result.split('<sys> ')[1]

'잘될 것 같아서 좋죠.” '